In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import json
from pandas.io.json import json_normalize

import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from plotly import tools
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

/Users/nguyenkhacbaoanh/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:45: FutureWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS will be built by the Apple Clang compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you won't need to install the gcc compiler anymore.
Instead of that, you'll need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", FutureWarning)


In [2]:
import os
os.listdir("input/")

['test.csv', 'train.csv', 'sample_submission.csv']

In [3]:
train = pd.read_csv("input/train.csv", dtype={"fullVisitorId": "str"})
test = pd.read_csv("input/test.csv", dtype={"fullVisitorId": "str"})

In [4]:
train.head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,Organic Search,20160902,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600


In [6]:
type(train.fullVisitorId[0])

str

In [7]:
columns_json = ['device', 'geoNetwork', 'totals', 'trafficSource']
dt_test = train.head(5)
dt_test

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,Organic Search,20160902,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600


In [16]:
dt_test.shape

(5, 12)

In [11]:
def load_data(path, nrows=None):
    columns_json = ['device', 'geoNetwork', 'totals', 'trafficSource']
    df = pd.read_csv(
                        path, 
                        converters={column: json.loads for column in columns_json}, 
                        dtype={'fullVisitorId':'str'})
    for column in columns_json:
        df_columns_json = json_normalize(df[column])
        df_columns_json.columns = [f"{column}.{subcolumn}" for subcolumn in df_columns_json.columns]
        df = df.drop(column, axis=1).merge(df_columns_json, left_index=True, right_index=True)
    return df

In [12]:
%%time
train = load_data("input/train.csv")
test = load_data("input/test.csv")

CPU times: user 6min 33s, sys: 4min 59s, total: 11min 32s
Wall time: 13min 9s


In [14]:
train.to_csv("input/train_extend.csv", index=False)
test.to_csv("input/test_extend.csv", index=False)

In [19]:
train.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [16]:
test.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20171016,6167871330617112363,6167871330617112363_1508151024,Not Socially Engaged,1508151024,2,1508151024,Chrome,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google
1,Organic Search,20171016,0643697640977915618,0643697640977915618_1508175522,Not Socially Engaged,1508175522,1,1508175522,Chrome,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,Organic Search,20171016,6059383810968229466,6059383810968229466_1508143220,Not Socially Engaged,1508143220,1,1508143220,Chrome,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
3,Organic Search,20171016,2376720078563423631,2376720078563423631_1508193530,Not Socially Engaged,1508193530,1,1508193530,Safari,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
4,Organic Search,20171016,2314544520795440038,2314544520795440038_1508217442,Not Socially Engaged,1508217442,1,1508217442,Safari,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google


In [17]:
print("train shape: ", train.shape)
print("test shape: ", test.shape)

train shape:  (903653, 55)
test shape:  (804684, 53)


trainset have 55 column while having 53 in testset. We knew that totals.transactionRevenue in training set is our target. We should find a column different between two set and drop it in training set

In [34]:
target = train['totals.transactionRevenue']
col_drop = set(train.columns).difference(set(test.columns))
print(col_drop)

train = train.drop(col_drop, axis=1)

{'totals.transactionRevenue', 'trafficSource.campaignCode'}


In [36]:
print("train shape: ", train.shape)
print("test shape: ", test.shape)
print("target shape: ", target.shape)

train shape:  (903653, 53)
test shape:  (804684, 53)
target shape:  (903653,)


## Try to predict log(transactionRevenue) by training model using lightgbm

In [18]:
train = pd.read_csv("input/train_extend.csv", dtype={'fullVisitorId':'str'})
test = pd.read_csv("input/test_extend.csv", dtype={'fullVisitorId':'str'})

/Users/nguyenkhacbaoanh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
target = train['totals.transactionRevenue']
col_drop = set(train.columns).difference(set(test.columns))
print(col_drop)

train = train.drop(col_drop, axis=1)

{'trafficSource.campaignCode', 'totals.transactionRevenue'}


In [21]:
target = np.log1p(target.fillna(0).astype('float64'))

drop all of constants columns in training set

In [22]:
const_cols = [c for c in train.columns if train[c].nunique(dropna=False) == 1]
const_cols

['socialEngagementType',
 'device.browserSize',
 'device.browserVersion',
 'device.flashVersion',
 'device.language',
 'device.mobileDeviceBranding',
 'device.mobileDeviceInfo',
 'device.mobileDeviceMarketingName',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.operatingSystemVersion',
 'device.screenColors',
 'device.screenResolution',
 'geoNetwork.cityId',
 'geoNetwork.latitude',
 'geoNetwork.longitude',
 'geoNetwork.networkLocation',
 'totals.visits',
 'trafficSource.adwordsClickInfo.criteriaParameters']

In [23]:
train = train.drop(const_cols, axis=1)
test = test.drop(const_cols, axis=1)

In [60]:
print("train shape: ", train.shape)
print("test shape: ", test.shape)

train shape:  (903653, 34)
test shape:  (804684, 34)


### Checking missing value

In [57]:
def check_missing_dt(train, test):
    missing_train = train.isnull().sum().values # array of all number of missing value per column
    missing_test = test.isnull().sum().values
    missing_train_per = missing_train/train.shape[0] # calculate percentage of missing value on total samples
    missing_test_per = missing_test/test.shape[0]
    return pd.DataFrame({"missing value's percentage of training": missing_train_per, 
                        "missing value's percentage of testing": missing_test_per}, 
                        index=train.columns).sort_values(
                                by=["missing value's percentage of training", "missing value's percentage of testing"], 
                                ascending=False)

In [58]:
missing_data = check_missing_dt(train, test)
missing_data

,missing value's percentage of training,missing value's percentage of testing
trafficSource.adContent,0.987887,0.933153
trafficSource.adwordsClickInfo.adNetworkType,0.976252,0.933124
trafficSource.adwordsClickInfo.isVideoAd,0.976252,0.933124
trafficSource.adwordsClickInfo.page,0.976252,0.933124
trafficSource.adwordsClickInfo.slot,0.976252,0.933124
trafficSource.adwordsClickInfo.gclId,0.976140,0.933064
trafficSource.isTrueDirect,0.696781,0.676254
trafficSource.referralPath,0.633774,0.707558
trafficSource.keyword,0.556551,0.485945
totals.bounces,0.501324,0.476878


In [27]:
train.head()

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,1472830385,1,1472830385,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,1472880147,1,1472880147,Firefox,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,1472865386,1,1472865386,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,1472881213,1,1472881213,UC Browser,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,1472822600,2,1472822600,Chrome,mobile,True,...,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google


Just the way that i struggle to seperate categorical columns and numerical columns

In [31]:
cat_cols = []
num_cols = []
for c in train.columns:
    try:
        train[c].astype(np.float64)
        num_cols.append(str(c))
    except ValueError:
        cat_cols.append(str(c))
print("categorical columns: ",cat_cols)
print()
print("numerical columns:",num_cols)

categorical columns:  ['channelGrouping', 'device.browser', 'device.deviceCategory', 'device.operatingSystem', 'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region', 'geoNetwork.subContinent', 'trafficSource.adContent', 'trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.gclId', 'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign', 'trafficSource.keyword', 'trafficSource.medium', 'trafficSource.referralPath', 'trafficSource.source']

numerical columns: ['date', 'fullVisitorId', 'sessionId', 'visitId', 'visitNumber', 'visitStartTime', 'device.isMobile', 'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews', 'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.adwordsClickInfo.page', 'trafficSource.isTrueDirect', 'totals.transactionRevenue']


**As we see on the numerical columns, there are some of columns classify like a numerical but naturelly these columns treat to reponse Yes No question, these columns are:**
1. device.isMobile
2. trafficSource.adwordsClickInfo.isVideoAd
3. trafficSource.adwordsClickInfo.page
4. trafficSource.isTrueDirect

------------
* fullVisitorId
* sessionId
* VisitId

*SessionId* **and** *VisitId* **combine to** *fullVisitorId*, **but fullVisitorId is our target to predict transactionRevenue, it was be formated like string**

------------
**Because we will split data training based on time, so** *VisitStartTime* **will be drop from the numerical columns**

------------
**and finally,** *totals.transactionRevenue* **is our output, it should be split into seperated dataframe** ***target***


In [39]:
num_cols = ['visitNumber', 
            'visitStartTime', 
            'totals.bounces', 
            'totals.hits', 
            'totals.newVisits', 
            'totals.pageviews']
for c in ['device.isMobile', 
                'trafficSource.adwordsClickInfo.isVideoAd', 
                'trafficSource.adwordsClickInfo.page', 
                'trafficSource.isTrueDirect']:
    cat_cols.append(c)
print(cat_cols)

['channelGrouping', 'device.browser', 'device.deviceCategory', 'device.operatingSystem', 'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region', 'geoNetwork.subContinent', 'trafficSource.adContent', 'trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.gclId', 'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign', 'trafficSource.keyword', 'trafficSource.medium', 'trafficSource.referralPath', 'trafficSource.source', 'device.isMobile', 'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.adwordsClickInfo.page', 'trafficSource.isTrueDirect']


    the function to plot the categorical columns

In [14]:
device = train['device.browser'].value_counts()[:6]
print(device.values)
trace = go.Bar(
            x= device.index,
            y= device.values,
            showlegend=True,
            marker = dict(
                color='rgba(255,0,255,0.8)'
            )
)
iplot([trace], filename="Device Browser")

[620364 182245  37069  19375  10205   7865]


In [26]:
def plot_cat(data, color, type_plot=None):
    if type_plot=='Bar':
        trace = go.Bar(
            x = data.index[::-1],
            y = data.values[::-1],
            showlegend=False,
            marker = dict(
                color=color
            )
        )
    elif type_plot=='Scatter':
        trace = go.Scatter(
            x = data.index[::-1],
            y = data.values[::-1],
            showlegend=False,
            marker = dict(
                color= color
            )
        )
    else:
        print("choose your plot type")
        return
    return trace

**Show 3 subcolumns of device column: **
* device.browser
* device.deviceCategory
* device.operatingSystem

> We explose each feature by our target that is totals.transactionRevenue

In [37]:
#browser
browser = train.groupby('device.browser')['totals.transactionRevenue'].agg(['size','count'])
browser.columns = ["count", "count of non-zero revenue"]
browser = browser.sort_values(by="count", ascending=False)
trace1 = plot_cat(browser["count"].head(6), 'rgba(60,70,115,0.5)', 'Bar')
trace2 = plot_cat(browser["count of non-zero revenue"].head(6), 'rgba(210,157,37,0.9)', 'Bar')
#deviceCategory
category = train.groupby("device.deviceCategory")["totals.transactionRevenue"].agg(['size', 'count'])
category.columns = ["count", "count of non-zero revenue"]
category = category.sort_values(by="count", ascending=False)
trace3 = plot_cat(category["count"].head(8), 'rgba(155,155,13,0.5)', 'Bar')
trace4 = plot_cat(category["count of non-zero revenue"].head(8), 'rgba(210,30,222,0.5)', 'Bar')
#operatingSystem
operating = train.groupby("device.operatingSystem")["totals.transactionRevenue"].agg(['size', 'count'])
operating.columns = ["count", "count of non-zero revenue"]
operating = operating.sort_values(by="count", ascending=False)
trace5 = plot_cat(operating["count"].head(8), 'rgba(100,90,80,0.5)', 'Bar')
trace6 = plot_cat(operating["count of non-zero revenue"].head(8), 'rgba(60,99,123,0.5)', 'Bar')
fig = tools.make_subplots(rows=3, cols=2, subplot_titles=["Browser - count", "Browser - Non Zero Revenue",
                                                          "Category - count", "Category - Non Zero Revenue",
                                                          "Operating System - count", "Operating System - Non Zero Revenue"
                                                         ])
fig.append_trace(trace1, 1,1)
fig.append_trace(trace2, 1,2)
fig.append_trace(trace3, 2,1)
fig.append_trace(trace4, 2,2)
fig.append_trace(trace5, 3,1)
fig.append_trace(trace6, 3,2)
fig['layout'].update(height=1300, width=900, paper_bgcolor='rgb(233,233,233)', title="Device Plots")
iplot(fig, filename="device-plots")

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5 ]  [ (3,2) x6,y6 ]



In [40]:
import datetime
train['date'] = train['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))

In [41]:
train['date'].head()

0    2016-09-02
1    2016-09-02
2    2016-09-02
3    2016-09-02
4    2016-09-02
Name: date, dtype: object

In [58]:
date = train.groupby('date')['totals.transactionRevenue'].agg(['size', 'count'])
date.columns = ["count", "count of non-zero Revenue"]
date = date.sort_index()
trace1 = plot_cat(date["count"], 'red', 'Scatter')
trace2 = plot_cat(date["count of non-zero Revenue"], 'green', 'Scatter')
fig = tools.make_subplots(rows=2, cols=1, subplot_titles=["count", "count of non zero revenue"])
fig['layout'].update(height=800, width=1000, title="Date plots", paper_bgcolor='rgba(233,233,233,0.4)')
fig.append_trace(trace1, 1,1)
fig.append_trace(trace2, 2, 1)
iplot(fig, filename="date-plots")

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [6]:
num_visit_per_id = train.groupby("fullVisitorId")["visitNumber"].agg(['size', 'count'])
num_visit_per_id.head()

,size,count
fullVisitorId,,
4823595352351,1,1
5103959234087,1,1
10278554503158,1,1
20424342248747,1,1
26722803385797,1,1


In [27]:
num_visit_per_id = train.groupby("fullVisitorId")["visitNumber"].agg(['size', 'count'])
num_visit_per_id.columns = ["count", "count of non zero revenue"]
num_visit_per_id.sort_values(by="count", ascending=False, inplace=True)
trace1 = plot_cat(num_visit_per_id["count"].head(20), 'rgba(145,210,50,0.5)', 'Bar')
trace2 = plot_cat(num_visit_per_id["count of non zero revenue"].head(20), 'rgba(145,210,50,0.5)', 'Bar')
fig = tools.make_subplots(rows=2, cols=1, subplot_titles=["Count", "Count of non zero revenue"])
fig.append_trace(trace1, 1,1)
fig.append_trace(trace2, 2,1)
fig['layout'].update(height=1200, width=900, paper_bgcolor='rgba(233,233,233,0.5)', title="Revenue per visitor id")
iplot(fig, filename="revenue-visitor")

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [42]:
from sklearn.preprocessing import LabelEncoder
for col in cat_cols:
    lbd = LabelEncoder()
    lbd.fit(list(train[col].values.astype('str')) + list(test[col].values.astype('str')))
    train[col] = lbd.transform(list(train[col].values.astype('str')))
    test[col] = lbd.transform(list(test[col].values.astype('str')))

In [43]:
for col in num_cols:
    train[col] = train[col].astype('float')
    test[col] = test[col].astype('float')

In [44]:
print(train.shape)
print(test.shape)

(903652, 35)
(804684, 34)


In [54]:
train["totals.transactionRevenue"].fillna(0, inplace=True)
y_train = train["totals.transactionRevenue"].values
drop_col = set(train.columns).difference(set(cat_cols + num_cols))
drop_col = list(drop_col) + ["VisitStartTime"]
drop_col

['sessionId',
 'fullVisitorId',
 'date',
 'totals.transactionRevenue',
 'visitId',
 'VisitStartTime']

In [67]:
# split the train dataset into train and valid based on time
X_train_df = train[train["date"]<=datetime.date(2017,5,31)]
X_val_df = train[train["date"]>datetime.date(2017,5,31)]
y_train = np.log1p(X_train_df["totals.transactionRevenue"].values)
y_val = np.log1p(X_val_df["totals.transactionRevenue"].values)

In [74]:
X_train = X_train[num_cols + cat_cols]
X_val = X_val[num_cols + cat_cols]

In [70]:
data_train = lgb.Dataset(X_train, label=y_train)
data_val = lgb.Dataset(X_val, label=y_val)

In [58]:
param = {
         'objective':'regression',
         'metric': 'rmse',
         'learning_rate':0.05,
         'num_leaves':100,
         'min_data_in_leaf':150,
         'max_depth':-1,
         'bagging_fraction':0.7,
         'feature_fraction':0.7,
         'max_bin':200,
         'random_state': 42}

In [71]:
model = lgb.train(param, data_train, 1000)

In [72]:
pred_y = model.predict(X_val, num_iteration=model.best_iteration)

In [75]:
len(pred_y)

137946

In [76]:
X_val.shape

(137946, 30)

In [73]:
pred_y[pred_y<0] = 0
val_pred_df = pd.DataFrame({"fullVisitorId":X_val_df["fullVisitorId"].values})
val_pred_df["transactionRevenue"] = X_val_df["totals.transactionRevenue"].values
val_pred_df["PredictedRevenue"] = np.expm1(pred_y)

val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
from sklearn.metrics import mean_squared_error
print(np.sqrt(mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), 
                                 np.log1p(val_pred_df["PredictedRevenue"].values))))

0.34216415185384363


In [85]:
train.to_csv("input/train_work.csv", index=False)
test.to_csv("input/test_work.csv", index=False)
target.to_csv("input/target_work.csv", index=False)

In [17]:
train = pd.read_csv("input/train_work.csv", dtype={'fullVisitorId':'str'})
test = pd.read_csv("input/test_work.csv", dtype={'fullVisitorId':'str'})
target = pd.read_csv("input/target_work.csv")

In [17]:
train.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device.browser',
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source'],
      dtype='object')

In [18]:
train = train.join(target, how='right')

In [19]:
train.rename(columns={'0.0': 'totals.transactionRevenue'}, inplace=True)

In [21]:
train["fullVisitorId"][0]

'1131660440785968503'